In [18]:
import nltk
import joblib
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
import warnings
warnings.filterwarnings('ignore')


In [3]:
dataset=pd.read_csv("C:\\Users\\ajlad\Downloads\\bbc_data.csv")

In [4]:
dataset

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment
...,...,...
2220,Warning over Windows Word files Writing a Mic...,tech
2221,Fast lifts rise into record books Two high-sp...,tech
2222,Nintendo adds media playing to DS Nintendo is...,tech
2223,Fast moving phone viruses appear Security fir...,tech


In [5]:
dataset.isnull().sum()

data      0
labels    0
dtype: int64

In [6]:
dataset.describe()

,data,labels
count,2225,2225
unique,2127,5
top,"Millions to miss out on the net By 2025, 40% ...",sport
freq,2,511


In [7]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [8]:
x=dataset["data"].apply(preprocess)
y=dataset["labels"]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x=vectorizer.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [19]:
# Apply PCA
pca = PCA(n_components=100, svd_solver='arpack')  # Retain 95% of variance
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

rf=RandomForestClassifier()

# Train and evaluate the SVM model with PCA
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2)
grid_search.fit(x_train_pca, y_train)




Fitting 5 folds for each of 576 candidates, totalling 2880 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150, 200]},
             verbose=2)

In [22]:
# Calculate accuracy
y_pred=grid_search.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best model: {accuracy:.4f}")

Accuracy of the best model: 0.9640


In [26]:
new_text = ['recently samsung launched galaxy 25 ultra']
prepo = [preprocess(text) for text in new_text]
vector = vectorizer.transform(prepo)
test_pca = pca.transform(vector)
print("RANDOM FOREST OUTPUT:")
result = grid_search.predict(test_pca)
print(result)

RANDOM FOREST OUTPUT:
['entertainment']
